In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time
import random
from IPython.display import Audio
import IPython.display as display

/home/pi/berryconda3/envs/tens/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.4 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/pi/berryconda3/envs/tens/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: builtins.type size changed, may indicate binary incompatibility. Expected 432, got 412
  return f(*args, **kwds)


In [2]:
class dynamic_object:
    pass
class datasource:
    def __init__(self, file_path, seq_len):
        self.seq_len = seq_len
        source = pd.read_csv(file_path)
        self.all_data = dynamic_object()
        self.train = dynamic_object()
        self.test = dynamic_object()
        self.all_data.x = np.vstack([source['dist'],
                     source['ax'],
                     source['ay'],
                     source['az'],
                     source['gx'],
                     source['gy'],
                     source['gz'],
                     source["shake"]]).T
        self.all_data.y = np.array(source['state']).reshape((-1, 1))
        self.all_data.m = source["dist"].shape[0]
        thre = int(self.all_data.m * 0.7)
        self.train.x = self.all_data.x[0:thre, :]
        self.train.y = self.all_data.y[0:thre, :]
        self.train.m = thre
        self.test.x = self.all_data.x[thre:, :]
        self.test.y = self.all_data.y[thre:, :]
        self.test.m = self.all_data.m - thre
    
    def sample_plt(self):
        bx, by = self.next_batch(2)
        plt.figure(dpi=100)
        plt.plot(by[0], linewidth=3)
        plt.plot(bx[0, :, 0], linewidth=1.2)
        plt.plot(bx[0, :, 1], linewidth=1.2)
        plt.plot(bx[0, :, 2], linewidth=0.5, linestyle='--')
        plt.plot(bx[0, :, 3], linewidth=0.5, linestyle='--')
        plt.plot(bx[0, :, 4], linewidth=0.5, linestyle='--')
        plt.plot(bx[0, :, 5], linewidth=0.5, linestyle='--')
        plt.plot(bx[0, :, 6], linewidth=1, linestyle='--')
        plt.legend(["dist", "ax", "ay", "az", "gx", "gy", "gz", "shake"])
        plt.title("Inputs and correct reaction")
        plt.show()
    
            
    def next_batch(self, batch_size, tp="train"):
        if tp == "train":
            maxin = self.train.m - self.seq_len
            ind = random.randint(0, maxin)
            batch_y = self.train.y[ind:ind + self.seq_len, :]
            batch_x = self.train.x[ind:ind + self.seq_len, :]
            for i in range(batch_size - 1):
                ind = random.randint(0, maxin)
                batch_x = arr_stack(self.train.x[ind:ind + self.seq_len, :], batch_x)
                batch_y = arr_stack(self.train.y[ind:ind + self.seq_len, :], batch_y)
            return batch_x, batch_y.reshape((batch_size, self.seq_len))
        else:
            maxin = self.test.m - self.seq_len
            ind = random.randint(0, maxin)
            batch_y = self.test.y[ind:ind + self.seq_len, :]
            batch_x = self.test.x[ind:ind + self.seq_len, :]
            for i in range(batch_size - 1):
                ind = random.randint(0, maxin)
                batch_y = arr_stack(self.test.y[ind:ind + self.seq_len, :], batch_y)
                batch_x = arr_stack(self.test.x[ind:ind + self.seq_len, :], batch_x)
            return batch_x, batch_y.reshape((batch_size, self.seq_len))

def arr_stack(a, b):
    if len(a.shape) == 2 and len(b.shape) == 2:
        return np.stack((a, b))
    shape = None
    if len(a.shape) > 2:
        shape = a.shape
    else:
        shape = b.shape
    return np.append(a, b).reshape((shape[0] + 1, shape[1], shape[2]))
    
    

In [3]:
# Define variables
batch_size = 56

tf.reset_default_graph()

x = tf.placeholder("float", [None, 96, 8])
y = tf.placeholder("float", [None, 96])

Wh = tf.Variable(tf.random_normal([8, 64]))
bh = tf.Variable(tf.random_normal([64]))
Wo = tf.Variable(tf.random_normal([64, 1]))
bo = tf.Variable(tf.random_normal([1]))

lstm_cell = tf.nn.rnn_cell.LSTMCell(64, forget_bias=0, state_is_tuple=True)
train_init = lstm_cell.zero_state(batch_size, "float")

# Build graph
x_flat = tf.reshape(x, [-1, 8])
a_flat = tf.tanh(tf.matmul(x_flat, Wh) + bh)
a = tf.reshape(a_flat, [-1, 96, 64])


outputs, state = tf.nn.dynamic_rnn(lstm_cell, a, initial_state=train_init)
# print(tf.nn.dynamic_rnn(lstm_cell, a, initial_state=train_init))

s_flat = tf.reshape(outputs, [-1, 64])
y_flat = tf.sigmoid(tf.matmul(s_flat, Wo) + bo)
y_ = tf.reshape(y_flat, [-1, 96])

cost = tf.reduce_mean((y - y_) * (y - y_))
train_step = tf.train.AdamOptimizer(0.0005).minimize(cost)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


In [4]:
dataset = datasource("master.csv", 96)
sess.run(tf.initialize_all_variables())

FileNotFoundError: File b'master.csv' does not exist

In [ ]:
%%time
dataset = datasource("master.csv", 96)
sess = tf.Session()
sess.run(tf.initialize_all_variables())
for i in range(5200):
    bx, by = dataset.next_batch(56)
    sess.run(train_step, feed_dict={x:bx, y:by})
    if i % 50 == 0:
        tx, ty = dataset.next_batch(56, tp="test")
        print("now %dth \t cost: %.4f\t vail:%.4f" % (i, sess.run(cost, feed_dict={x:bx, y:by}), sess.run(cost, feed_dict={x:tx, y:ty})))

now 0th 	 cost: 0.1761	 vail:0.1263
now 50th 	 cost: 0.1136	 vail:0.0803
now 100th 	 cost: 0.0763	 vail:0.0870
now 150th 	 cost: 0.0432	 vail:0.0558
now 200th 	 cost: 0.0249	 vail:0.0637
now 250th 	 cost: 0.0168	 vail:0.0653
now 300th 	 cost: 0.0208	 vail:0.0834
now 350th 	 cost: 0.0169	 vail:0.0612
now 400th 	 cost: 0.0144	 vail:0.0639
now 450th 	 cost: 0.0108	 vail:0.0593
now 500th 	 cost: 0.0120	 vail:0.0760
now 550th 	 cost: 0.0083	 vail:0.0689
now 600th 	 cost: 0.0121	 vail:0.0667
now 650th 	 cost: 0.0097	 vail:0.0641
now 700th 	 cost: 0.0111	 vail:0.0599
now 750th 	 cost: 0.0091	 vail:0.0785
now 800th 	 cost: 0.0076	 vail:0.0875
now 850th 	 cost: 0.0097	 vail:0.0833
now 900th 	 cost: 0.0052	 vail:0.0859
now 950th 	 cost: 0.0109	 vail:0.0701
now 1000th 	 cost: 0.0083	 vail:0.0762
now 1050th 	 cost: 0.0060	 vail:0.0822
now 1100th 	 cost: 0.0082	 vail:0.0835
now 1150th 	 cost: 0.0081	 vail:0.0783
now 1200th 	 cost: 0.0086	 vail:0.0863
now 1250th 	 cost: 0.0077	 vail:0.0841
now 1300t

In [ ]:
Audio(url="http://fjdx.sc.chinaz.com/Files/DownLoad/sound1/201702/8378.mp3", autoplay=True)

In [ ]:
tx, ty = dataset.next_batch(56, tp="test")
yp = sess.run(y_, feed_dict={x:tx})
tx = tx[0, :]
ty = ty[0, :]
plt.figure(dpi=100)
plt.plot(ty, linewidth=3)
plt.plot(yp[0, :], linewidth=3)
plt.plot(tx[:, 0], linewidth=1.2)
plt.plot(tx[:, 1], linewidth=1.2)
plt.legend(["dist", "ans", "ay", "az", "gx", "gy", "gz", "shake"])
plt.title("Inputs and correct reaction")
plt.show()

In [7]:

sess = tf.Session()
saver = tf.train.Saver(write_version=tf.train.SaverDef.V2)
saver.restore(sess, "./para")

INFO:tensorflow:Restoring parameters from ./para


InvalidArgumentError: Restoring from checkpoint failed. This is most likely due to a mismatch between the current graph and the graph from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Assign requires shapes of both tensors to match. lhs shape= [8,64] rhs shape= [8,32]
	 [[node save_2/Assign (defined at <ipython-input-7-85fa9704e879>:3) ]]

Caused by op 'save_2/Assign', defined at:
  File "/home/pi/berryconda3/envs/tens/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/pi/berryconda3/envs/tens/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/pi/berryconda3/envs/tens/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/pi/berryconda3/envs/tens/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/pi/be